In [1]:
import os
import pandas as pd
import nltk
import pickle as pkl
import re, string
#import unicodedata
import gensim
from gensim import corpora, models, similarities 
from collections import Counter
from sklearn import datasets


C:\Users\thaidao\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


# Loading the data

I am using the BBC raw text dataset available at http://mlg.ucd.ie/datasets/bbc.html. You can download and unzip to 'bbc' folder.

In [2]:
DATA_DIR = os.getcwd()+'\\bbc'  #Windows's dir delimiter
#subdir are CATEGORIES = ['business', 'entertainment', 'politics', 'sport', 'tech']
CATEGORIES = os.listdir(path='./bbc')

#print(DATA_DIR)
print(CATEGORIES)

['business', 'entertainment', 'politics', 'sport', 'tech']


In [ ]:
#DATA_DIR


In [3]:
#This will load all articles into a list of articles.
raw_text = datasets.load_files(DATA_DIR,
    categories=CATEGORIES, encoding="Latin-1")

In [4]:
#Let see the content of few articles
raw_text.data[0]


'Tate & Lyle boss bags top award\n\nTate & Lyle\'s chief executive has been named European Businessman of the Year by a leading business magazine.\n\nIain Ferguson was awarded the title by US publication Forbes for returning one of the UK\'s "venerable" manufacturers to the country\'s top 100 companies. The sugar group had been absent from the FTSE 100 for seven years until Mr Ferguson helped it return to growth. Tate\'s shares have leapt 55% this year, boosted by firming sugar prices and sales of its artificial sweeteners.\n\n"After years of a sagging stock price and a seven-year hiatus from the FTSE 100, one of Britain\'s venerable manufacturers has returned to the vaunted index," Forbes said. Mr Ferguson took the helm at the company in 2003, after spending most of his career at consumer goods giant Unilever. Tate & Lyle, which was an original member of the historic FT-30 index in 1935, operates more than 41 factories and 20 more additional production facilities in 28 countries. Prev

In [5]:
raw_text.target_names[0]

'business'

In [6]:
raw_text.data[1]

"Halo 2 sells five million copies\n\nMicrosoft is celebrating bumper sales of its Xbox sci-fi shooter, Halo 2.\n\nThe game has sold more than five million copies worldwide since it went on sale in mid-November, the company said. Halo 2 has proved popular online, with gamers notching up a record 28 million hours playing the game on Xbox Live. According to Microsoft, nine out of 10 Xbox Live members have played the game for an average of 91 minutes per session.\n\nThe sequel to the best-selling Need for Speed: Underground has inched ahead of the competition to take the top slot in the official UK games charts. The racing game moved up one spot to first place, nudging GTA: San Andreas down to second place. Halo 2 dropped one place to five, while Half-Life 2 fell to number nine. Last week's new releases, GoldenEye: Rogue Agent and Killzone, both failed to make it into the top 10, debuting at number 11 and 12 respectively.\n\nRecord numbers of Warcraft fans are settling in the games online 

In [7]:
raw_text.target_names[1]


'entertainment'

In [8]:
#there are 2225 text/news extracted, matches the dataset total files
len(raw_text.data)

2225

In [9]:
#Save all text files to .plk for faster reload next time
pkl.dump(raw_text.data, open('raw_text.pkl','wb'))


In [10]:
raw_text_df = pd.DataFrame(raw_text.data,columns=['raw_text'])

In [11]:
raw_text.data[1]

"Halo 2 sells five million copies\n\nMicrosoft is celebrating bumper sales of its Xbox sci-fi shooter, Halo 2.\n\nThe game has sold more than five million copies worldwide since it went on sale in mid-November, the company said. Halo 2 has proved popular online, with gamers notching up a record 28 million hours playing the game on Xbox Live. According to Microsoft, nine out of 10 Xbox Live members have played the game for an average of 91 minutes per session.\n\nThe sequel to the best-selling Need for Speed: Underground has inched ahead of the competition to take the top slot in the official UK games charts. The racing game moved up one spot to first place, nudging GTA: San Andreas down to second place. Halo 2 dropped one place to five, while Half-Life 2 fell to number nine. Last week's new releases, GoldenEye: Rogue Agent and Killzone, both failed to make it into the top 10, debuting at number 11 and 12 respectively.\n\nRecord numbers of Warcraft fans are settling in the games online 

# Preprocessing steps:
*  Basic text cleaning:  Lowering letter cases, and using alphabet without special characters.
*  Filtering out Stop Words
*  Stemming 
*  Or more later....


In [12]:
clean_list= re.sub("[^a-zA-Z]", " ", raw_text.data[1].lower().strip())


In [13]:
clean_list.split(" ")

['halo',
 '',
 '',
 'sells',
 'five',
 'million',
 'copies',
 '',
 'microsoft',
 'is',
 'celebrating',
 'bumper',
 'sales',
 'of',
 'its',
 'xbox',
 'sci',
 'fi',
 'shooter',
 '',
 'halo',
 '',
 '',
 '',
 '',
 'the',
 'game',
 'has',
 'sold',
 'more',
 'than',
 'five',
 'million',
 'copies',
 'worldwide',
 'since',
 'it',
 'went',
 'on',
 'sale',
 'in',
 'mid',
 'november',
 '',
 'the',
 'company',
 'said',
 '',
 'halo',
 '',
 '',
 'has',
 'proved',
 'popular',
 'online',
 '',
 'with',
 'gamers',
 'notching',
 'up',
 'a',
 'record',
 '',
 '',
 '',
 'million',
 'hours',
 'playing',
 'the',
 'game',
 'on',
 'xbox',
 'live',
 '',
 'according',
 'to',
 'microsoft',
 '',
 'nine',
 'out',
 'of',
 '',
 '',
 '',
 'xbox',
 'live',
 'members',
 'have',
 'played',
 'the',
 'game',
 'for',
 'an',
 'average',
 'of',
 '',
 '',
 '',
 'minutes',
 'per',
 'session',
 '',
 '',
 'the',
 'sequel',
 'to',
 'the',
 'best',
 'selling',
 'need',
 'for',
 'speed',
 '',
 'underground',
 'has',
 'inched',
 'ahea

In [14]:
stopwords = nltk.corpus.stopwords.words('english')
print(sorted(stopwords))


['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', 'd', 'did', 'didn', 'do', 'does', 'doesn', 'doing', 'don', 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', 'has', 'hasn', 'have', 'haven', 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', 'more', 'most', 'mustn', 'my', 'myself', 'needn', 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', 'she', 'should', 'shouldn', 'so', 'some', 'such', 't', 'than', 'that', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'v

In [15]:
wcount = Counter()
filtered_words= [word for word in clean_list[2].split() if word not in stopwords and len(word) > 2]  
wcount.update(filtered_words)
print(wcount.most_common(25))


[]


In [17]:
stopwords = nltk.corpus.stopwords.words('english') 
stemmer = nltk.stem.snowball.SnowballStemmer('english')

def news_to_tokenized_words (text):
    #  Basic text cleaning:  Lowering letter cases, and using alphabet without special characters.
    clean_text = re.sub("[^a-zA-Z]", " ", text.lower().strip())
    
    #  Filtering out Stop Words and words that are less than 3 chars
    filtered_words= [word for word in clean_text.split(" ") if word not in stopwords and len(word) > 2]

    #  Stemming 
    tokenized_words = [stemmer.stem(word) for word in filtered_words ]
    
    return tokenized_words

In [18]:
raw_text_df.insert(raw_text_df.shape[1], 'clean_text', raw_text_df.raw_text.apply(news_to_tokenized_words))

In [19]:
#The tokenized list looks ok
raw_text_df.clean_text[1]

['halo',
 'sell',
 'five',
 'million',
 'copi',
 'microsoft',
 'celebr',
 'bumper',
 'sale',
 'xbox',
 'sci',
 'shooter',
 'halo',
 'game',
 'sold',
 'five',
 'million',
 'copi',
 'worldwid',
 'sinc',
 'went',
 'sale',
 'mid',
 'novemb',
 'compani',
 'said',
 'halo',
 'prove',
 'popular',
 'onlin',
 'gamer',
 'notch',
 'record',
 'million',
 'hour',
 'play',
 'game',
 'xbox',
 'live',
 'accord',
 'microsoft',
 'nine',
 'xbox',
 'live',
 'member',
 'play',
 'game',
 'averag',
 'minut',
 'per',
 'session',
 'sequel',
 'best',
 'sell',
 'need',
 'speed',
 'underground',
 'inch',
 'ahead',
 'competit',
 'take',
 'top',
 'slot',
 'offici',
 'game',
 'chart',
 'race',
 'game',
 'move',
 'one',
 'spot',
 'first',
 'place',
 'nudg',
 'gta',
 'san',
 'andrea',
 'second',
 'place',
 'halo',
 'drop',
 'one',
 'place',
 'five',
 'half',
 'life',
 'fell',
 'number',
 'nine',
 'last',
 'week',
 'new',
 'releas',
 'goldeney',
 'rogu',
 'agent',
 'killzon',
 'fail',
 'make',
 'top',
 'debut',
 'number

# Training

In [21]:
dictionary = corpora.Dictionary(raw_text_df.clean_text)
print(dictionary)


Dictionary(18656 unique tokens: ['carrot', 'commiss', 'zombi', 'arrigo', 'plug']...)


In [22]:
corpus=[dictionary.doc2bow(doc) for doc in raw_text_df.clean_text]


In [ ]:
#Estimated memory req't to train: 8 bytes * num_terms * num_topics * 3 ==> 8*18656*4*3 = apprx. 1.7M
#Source:  https://github.com/RaRe-Technologies/gensim/wiki/Recipes-&-FAQ#q5-i-am-getting-out-of-memory-errors-with-lsi-how-much-memory-do-i-need


In [23]:
%%time
ldamodel=gensim.models.LdaMulticore(corpus, num_topics=len(CATEGORIES), id2word=dictionary)



Wall time: 14.2 s


In [24]:
print(ldamodel.print_topics(num_topics=len(CATEGORIES), num_words=50))

[(0, '0.017*"said" + 0.005*"one" + 0.004*"also" + 0.004*"year" + 0.004*"two" + 0.004*"make" + 0.003*"new" + 0.003*"time" + 0.003*"peopl" + 0.003*"use" + 0.003*"best" + 0.003*"would" + 0.003*"show" + 0.003*"includ" + 0.003*"get" + 0.002*"film" + 0.002*"last" + 0.002*"say" + 0.002*"british" + 0.002*"could" + 0.002*"play" + 0.002*"mani" + 0.002*"bbc" + 0.002*"take" + 0.002*"award" + 0.002*"week" + 0.002*"like" + 0.002*"compani" + 0.002*"music" + 0.002*"first" + 0.002*"told" + 0.002*"govern" + 0.002*"firm" + 0.002*"game" + 0.002*"three" + 0.002*"plan" + 0.002*"report" + 0.002*"number" + 0.002*"next" + 0.002*"call" + 0.002*"world" + 0.002*"way" + 0.002*"want" + 0.002*"month" + 0.002*"right" + 0.002*"group" + 0.002*"open" + 0.002*"home" + 0.002*"phone" + 0.002*"market"'), (1, '0.011*"said" + 0.007*"year" + 0.005*"also" + 0.005*"one" + 0.004*"would" + 0.004*"new" + 0.004*"last" + 0.004*"game" + 0.003*"peopl" + 0.003*"world" + 0.003*"could" + 0.003*"time" + 0.003*"make" + 0.003*"compani" + 0.0

In [26]:
topics = ldamodel.show_topics(formatted=False, num_words=100)
    
for t in range(len(topics)):
    print("\nTopic {}, top {} words:".format(t+1,100))
    print(" ".join([w[0] for w in topics[t][1]]))


Topic 1, top 100 words:
said one also year two make new time peopl use best would show includ get film last say british could play mani bbc take award week like compani music first told govern firm game three plan report number next call world way want month right group open home phone market look hope back second director win minist help technolog mobil player final current star countri system day user need issu made think set deal ad news expect well start microsoft good top public part follow move servic develop site inform perform network tri million intern place becom unit virus work

Topic 2, top 100 words:
said year also one would new last game peopl world could time make compani best two first say govern month sale music number market like come take work day use award set way includ need get countri want well nation ad see record firm made next look servic back second cost help end may mani top england play million star win price technolog player part industri good week offer 

# Result:  
 
Need to clean raw text more to include stopwords such as "the", "said", "would", "could" ect.  
One could infer from topic 1 that it is about technology, as the top words are microsoft, virus,
network, and technology.  Topic 5 could be government, as the top words include govern, elect, ministry, tori, ect


